In [2]:
import os
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch
import torchvision
from glob import glob
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import torchvision.transforms as transform
from torch.utils.data import DataLoader,Dataset

import sys
sys.path.append("/Users/nathanieljames/Desktop/dl_final_proj/dl_final_proj")
from utils.cityscapes import CityscapesDataset, get_loaders

/opt/homebrew/Caskroom/miniforge/base/envs/dl/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/dl/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/homebrew/Caskroom/miniforge/base/envs/dl/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <E459C462-F863-3A5A-AC9F-FD77B14BE845> /opt/homebrew/Caskroom/miniforge/base/envs/dl/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
dataloader, valloader = get_loaders(batch_size=32, subclass = "combined")
print(len(dataloader), len(valloader))

279 1502


In [4]:
dtype = torch.float
device = torch.device("mps")

In [13]:
#design a resnet based network to take in a (3,256,256) image and output a (3,256,256) image
# keep it around 1M trainiable parameter

class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.res1 = ResNetBlock(64, 64, 3, 1, 1)
        self.res2 = ResNetBlock(64, 64, 3, 1, 1)
        self.res3 = ResNetBlock(64, 64, 3, 1, 1)
        self.res4 = ResNetBlock(64, 64, 3, 1, 1)
        self.res5 = ResNetBlock(64, 64, 3, 1, 1)
        self.res6 = ResNetBlock(64, 64, 3, 1, 1)
        self.res7 = ResNetBlock(64, 64, 3, 1, 1)
        self.res8 = ResNetBlock(64, 64, 3, 1, 1)
        self.res9 = ResNetBlock(64, 64, 3, 1, 1)
        self.res10 = ResNetBlock(64, 64, 3, 1, 1)
        # turn back int 3x256x256
        self.conv2 = nn.Conv2d(64, 3, 3, 1, 1)

        

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.res1(out)
        out = self.res2(out)
        out = self.res3(out)
        out = self.res4(out)
        out = self.res5(out)
        out = self.res6(out)
        out = self.res7(out)
        out = self.res8(out)
        out = self.res9(out)
        out = self.res10(out)
        out = self.conv2(out)
        return out

In [14]:
model = ResNet().float()

from torchsummary import summary
summary(model, (3,256,256))
model = model.to(device)

epochs = 5

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

train_acc = []
val_acc = []
train_loss = []
val_loss = []

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4         [-1, 64, 256, 256]          36,928
       BatchNorm2d-5         [-1, 64, 256, 256]             128
              ReLU-6         [-1, 64, 256, 256]               0
            Conv2d-7         [-1, 64, 256, 256]          36,928
       BatchNorm2d-8         [-1, 64, 256, 256]             128
              ReLU-9         [-1, 64, 256, 256]               0
      ResNetBlock-10         [-1, 64, 256, 256]               0
           Conv2d-11         [-1, 64, 256, 256]          36,928
      BatchNorm2d-12         [-1, 64, 256, 256]             128
             ReLU-13         [-1, 64, 256, 256]               0
           Conv2d-14         [-1, 64, 2